In [38]:
# Feature Extraction with RFE 
from pandas import read_csv 
from sklearn.feature_selection import RFE 
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
print ("libraries loaded.")
#NOTE: pima-indians-diabetes.data.csv column 'class' is categorical data, not suitable for regression.

libraries loaded.


In [18]:
import os 
print (os.getcwd())


D:\2017_working\gitrepos\ML_time_series_training


In [56]:
# load data 
filename = 'pima-indians-diabetes.data.csv' 
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class'] 
dataframe = read_csv(filename, names=names) 
array = dataframe.values 
print (type(array), array.shape)

<class 'numpy.ndarray'> (768, 9)


In [57]:
dataframe.describe()

,preg,plas,pres,skin,test,mass,pedi,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [62]:
#print (dataframe['class'].unique())
print (names)
print ("Unique values")
for name in names:
    print(name, ":", dataframe[name].unique().shape[0])

['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
Unique values
preg : 17
plas : 136
pres : 47
skin : 51
test : 186
mass : 248
pedi : 517
age : 52
class : 2


In [33]:
X = array[:,0:8] 
Y = array[:,8] 
# feature extraction 
model = LogisticRegression() 
rfe = RFE(model, 3) #select 3 features
#http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html
#RFE(estimator, n_features_to_select=None, step=1, verbose=0)
fit = rfe.fit(X, Y) 
print ("model created and fitted.")
print (type(rfe))
print (type(fit))


model created and fitted.
<class 'sklearn.feature_selection.rfe.RFE'>
<class 'sklearn.feature_selection.rfe.RFE'>


In [21]:
fitFeatures = fit.n_features_
print("Num Features: %d" % fitFeatures) 
fitSupport = fit.support_
print (type(fitSupport), len(fitSupport))
print (type(fitSupport[0]), fitSupport[0])
print("Selected Features: %s" % fitSupport)
fitRanking = fit.ranking_
print("Feature Ranking: %s" % fitRanking)
print("column names :", names)

Num Features: 3
<class 'numpy.ndarray'> 8
<class 'numpy.bool_'> True
Selected Features: [ True False False False False  True  True False]
Feature Ranking: [1 2 3 5 6 1 1 4]
column names : ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']


In [22]:
fit.support_

array([ True, False, False, False, False,  True,  True, False], dtype=bool)

In [23]:
count = 0
selectedFeatures = []
for fit in fit.support_:
    if fit:
        selectedFeatures.append(names[count])
    count += 1
print("selectedFeatures:", selectedFeatures)

selectedFeatures: ['preg', 'mass', 'pedi']


In [29]:
# Create linear regression object
regr = linear_model.LinearRegression()
print (type(regr))

<class 'sklearn.linear_model.base.LinearRegression'>


In [37]:
regr.fit(X, Y)
y_pred = regr.predict(X)#should be splitting input data into train/test then predicting on test
coefficients = regr.coef_
print('Coefficients: \n', coefficients)
print (type(coefficients), coefficients.shape)
#
print("Mean squared error: %.2f" % mean_squared_error(Y, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y, y_pred))


Coefficients: 
 [ 0.02059187  0.00592027 -0.00233188  0.00015452 -0.00018053  0.01324403
  0.14723744  0.00262139]
<class 'numpy.ndarray'> (8,)
Mean squared error: 0.16
Variance score: 0.30


In [74]:
coeffNames = pd.DataFrame(coefficients)
#print ("coefficients:", type(coeffNames), coeffNames.shape, coeffNames)
names = pd.DataFrame(names[0:8])
#print ("names:", type(names), names.shape, names)
coeffNames = pd.concat([names, coeffNames], axis=1)
coeffNames.columns = ['names', 'coefficients']
#print (type(coeffNames), coeffNames.shape)
#print (coeffNames)
coeffNames['coefficientsABS'] = coeffNames.coefficients.abs()
#sort column by coefficients descending order.
#NBB: dataframe.sort does not sort by absolute values.
coeffNames.sort_values('coefficientsABS', ascending=False)

,names,coefficients,coefficientsABS
6,pedi,0.147237,0.147237
0,preg,0.020592,0.020592
5,mass,0.013244,0.013244
1,plas,0.005920,0.005920
7,age,0.002621,0.002621
2,pres,-0.002332,0.002332
4,test,-0.000181,0.000181
3,skin,0.000155,0.000155
